# Init

In [1]:
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F
import fiftyone.brain as fob

ModuleNotFoundError: No module named 'fiftyone.zoo'

In [7]:
# The directory to which to write the exported dataset
EXPORT_DIR = "test_export2"

# dataset_type = fo.types.COCODetectionDataset  # COCO
EXPORT_DATASET_TYPE = fo.types.dataset_types.YOLOv5Dataset # YOLOv5

# The name of the sample field containing the label that you wish to export
# Used when exporting labeled datasets (e.g., classification or detection)
LABEL_FIELD = "ground_truth"  # for example

CLASSES_OF_INTEREST = ["Motorcycle", "Car", "Truck"]

In [3]:
# List available zoo datasets
print('######### Available zoo dataset #########')
print(foz.list_zoo_datasets())
print()

print('######### Donwloaded zoo dataset #########')
fo.pprint(foz.list_downloaded_zoo_datasets())
print()

print('######### Dataset list #########')
print(fo.list_datasets())


######### Available zoo dataset #########
['bdd100k', 'caltech101', 'caltech256', 'cifar10', 'cifar100', 'cityscapes', 'coco-2014', 'coco-2017', 'fashion-mnist', 'hmdb51', 'imagenet-2012', 'imagenet-sample', 'kitti', 'lfw', 'mnist', 'open-images-v6', 'quickstart', 'quickstart-geo', 'quickstart-video', 'ucf101', 'voc-2007', 'voc-2012']

######### Donwloaded zoo dataset #########
{
    'coco-2017': (
        '/home/dhp/fiftyone/coco-2017',
    ),
    'open-images-v6': (
        '/home/dhp/fiftyone/open-images-v6',
    ),
}

######### Dataset list #########
['coco-2017-all', 'open-image-vehicle_aug']


In [4]:
from IPython.display import clear_output
session = fo.launch_app()
# session.open_tab()
clear_output()

# Delete dataset

In [ ]:
# # delete a dataset
# fo.delete_dataset(name='open-image-vehicle_aug')

# Load dataset

In [ ]:
# # # load a dataset from the zoo and save to database

# dataset = foz.load_zoo_dataset(
#     "open-images-v6",
#     splits=["test", 'validation'],
#     dataset_name="open-images-v6",
#     download_if_necessary=False,   
# )
# dataset_or_view = dataset
# dataset_or_view

In [5]:
# # Load existing dataset from the database

dataset = fo.load_dataset(name="open-image-vehicle_aug")
dataset_or_view = dataset
dataset_or_view

Name:        open-image-vehicle_aug
Media type:  image
Num samples: 58653
Persistent:  True
Tags:        ['train', 'train_aug', 'validation', 'validation_aug']
Sample fields:
    id:                   fiftyone.core.fields.ObjectIdField
    filepath:             fiftyone.core.fields.StringField
    tags:                 fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:             fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    ground_truth:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    uniqueness:           fiftyone.core.fields.FloatField
    lightness:            fiftyone.core.fields.FloatField
    mistakenness_eval_tp: fiftyone.core.fields.IntField
    mistakenness_eval_fp: fiftyone.core.fields.IntField
    mistakenness_eval_fn: fiftyone.core.fields.IntField

In [ ]:
# # # Load dataset from the directories

# name = "open-image-vehicle_aug"
# dataset_dir = "test_export"

# # The splits to load
# splits = ["train", "validation","train_aug", "validation_aug"]

# # Load the dataset, using tags to mark the samples in each split
# dataset = fo.Dataset(name)
# for split in splits:
#     dataset.add_dir(
#         dataset_dir=dataset_dir,
#         dataset_type=fo.types.YOLOv5Dataset,
#         split=split,
#         tags=split,
# )

# # View summary info about the dataset
# dataset_or_view = dataset
# dataset

In [ ]:
# dataset.default_classes

In [ ]:
# dataset.save()

In [ ]:
# make dataset persistent
# dataset.name = 'open-images-v6'
dataset.persistent = True

In [ ]:
view = dataset.filter_labels(LABEL_FIELD, F("label").is_in(CLASSES_OF_INTEREST))
# view = view.match_tags('validation')
dataset_or_view = view

In [7]:
session.view = dataset_or_view.view()
clear_output()

# remove duplicates

In [ ]:
import fiftyone.utils.eval as foue
from fiftyone import ViewField as F

IOU_THRES = 0.75

foue.compute_max_ious(dataset_or_view, LABEL_FIELD, attr_name="max_iou", classwise=True)

# Retrieve detections that overlap above a chosen threshold
dups_view = dataset_or_view.filter_labels(LABEL_FIELD, F("max_iou") >= IOU_THRES)
session.view = dups_view
clear_output()

In [ ]:
dir(dups_view)

In [ ]:
# After tagging in FiftyOne UI
print(dataset.count_label_tags())

In [ ]:
dataset.delete_labels(tags="duplicate")

# Verify that tagged labels were deleted
print(dataset.count_label_tags())

# Calculate uniqueness


In [ ]:
fob.compute_uniqueness(dataset_or_view)

In [ ]:
dataset.save()

In [ ]:
unique_view = dataset.sort_by("uniqueness")
session.view = unique_view
clear_output()

# Calculate similarity


In [ ]:
fob.compute_similarity(dataset_or_view, brain_key="image_sim",)

In [ ]:
dataset.save()

# Evaluate model

## Compute Hardness

In [ ]:
fob.compute_hardness(dataset, "predictions")
dataset.save()

In [ ]:
fob.compute_mistakenness(dataset, "ground_truth", label_field="ground_truth")
dataset.save()

# Export data and labels

In [ ]:
# dataset_or_view.export(
#     export_dir=EXPORT_DIR,
#     dataset_type=EXPORT_DATASET_TYPE,
#     label_field=LABEL_FIELD,
#     classes=CLASSES_OF_INTEREST,
# )

In [8]:
dataset.count_sample_tags()

{'train_aug': 28092,
 'validation_aug': 9380,
 'train': 15882,
 'validation': 5299}

In [21]:
dataset

Name:        coco-2017-all
Media type:  image
Num samples: 212446
Persistent:  True
Tags:        ['test', 'train', 'validation']
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.Metadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    uniqueness:   fiftyone.core.fields.FloatField

In [8]:
dataset_or_view = dataset

for split in [
        'train', 
        # 'test',
        'validation',
        'train_aug',
        'validation_aug',
        ]:
    split_view = dataset_or_view.match_tags(split)

    # Export the dataset
    split_view.export(
        export_dir=EXPORT_DIR,
        dataset_type=EXPORT_DATASET_TYPE,
        label_field=LABEL_FIELD,
        classes=CLASSES_OF_INTEREST,
        split=split,
    )

 100% |█████████████| 15882/15882 [2.0m elapsed, 0s remaining, 249.3 samples/s]      
Directory 'test_export2' already exists; export will be merged with existing files
 100% |███████████████| 5299/5299 [40.5s elapsed, 0s remaining, 212.2 samples/s]      
Directory 'test_export2' already exists; export will be merged with existing files
 100% |█████████████| 28092/28092 [3.3m elapsed, 0s remaining, 225.3 samples/s]      
Directory 'test_export2' already exists; export will be merged with existing files
 100% |███████████████| 9380/9380 [1.3m elapsed, 0s remaining, 223.9 samples/s]       


# Visualize data

In [ ]:
import numpy as np
import cv2

# Construct a `num_samples x num_pixels` array of images
images = np.array([
    cv2.imread(f, cv2.IMREAD_UNCHANGED).ravel()
    for f in dataset.values("filepath")
])

# Compute 2D embeddings
results = fob.compute_visualization(dataset, embeddings=images, seed=51)

# Visualize embeddings, colored by ground truth label
plot = results.visualize(labels="ground_truth.label")
plot.show(height=720)

session.plots.attach(plot)

# Augment data

In [ ]:
%%bash

# ! pip install albumentations tqdm

python augment_box.py\
    -si 'test_export/images/train'\
    -sl 'test_export/labels/train'\
    -di 'test_export/images/train_aug'\
    -dl 'test_export/labels/train_aug'\
    -n 2

python augment_box.py\
    -si 'test_export/images/validation'\
    -sl 'test_export/labels/validation'\
    -di 'test_export/images/validation_aug'\
    -dl 'test_export/labels/validation_aug'\
    -n 2

In [ ]:
fo.delete_dataset(name='open-image-vehicle_aug')

In [ ]:
name = "open-image-vehicle_aug"
dataset_dir = "test_export"

# The splits to load
splits = ["train", "train_aug", "validation", "validation_aug"]

# Load the dataset, using tags to mark the samples in each split
dataset = fo.Dataset(name)
for split in splits:
    dataset.add_dir(
        dataset_dir=dataset_dir,
        dataset_type=fo.types.YOLOv5Dataset,
        split=split,
        tags=split,
)

# View summary info about the dataset
dataset_or_view = dataset
dataset

# Add custom tag

## Lightness

In [ ]:
dataset.add_sample_field(field_name='lightness', ftype=fo.FloatField)
dataset

In [ ]:
import cv2
import numpy as np
from tqdm import tqdm

lightness_values = []

for f in tqdm(dataset.values("filepath")):
    img = cv2.imread(f)
    hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    v = np.mean(hsv[...,[2]])
    lightness_values.append(v)

In [ ]:
dataset.set_values('lightness', lightness_values)

In [ ]:
light_view = dataset.sort_by("lightness")
session.view = light_view
clear_output()

## Aggregate statistics (box area)

In [ ]:
dataset.compute_metadata()

# Expression that computes the area of a bounding box, in pixels
# Bboxes are in [top-left-x, top-left-y, width, height] format
bbox_width = F("bounding_box")[2] * F("$metadata.width")
bbox_height = F("bounding_box")[3] * F("$metadata.height")
bbox_area = bbox_width * bbox_height

# Expression that computes the area of ground truth bboxes
gt_areas = F("ground_truth.detections[]").apply(bbox_area)

# Compute (min, max, mean) of ground truth bounding boxes
print(dataset.bounds(gt_areas))
print(dataset.mean(gt_areas))

In [ ]:
dataset.default_classes

# Modify dataset and save to database

## Modify sample tags
See more: https://voxel51.com/docs/fiftyone/user_guide/using_views.html#tagging-contents

In [ ]:
# remove pre-existing tags
dataset.untag_samples("test") 

In [ ]:
# replace tag
dataset.match_tags("test").tag_samples("train")
dataset.match_tags("test").untag_samples("test")

In [ ]:
dataset

In [ ]:
# Perform a random 90-10 test-train split
dataset.take(0.1 * len(dataset)).tag_samples("test")
dataset.match_tags("test", bool=False).tag_samples("train")

print(dataset.count_sample_tags())
# {'train': 180, 'test': 20}

## Modify classes

In [ ]:
# lower all classes
# dataset.default_classes = [i.lower() for i in dataset.default_classes]
# dataset.default_classes[:3]


# TODO: add modify labels in the samples.

In [ ]:
dataset

In [ ]:
# save to database
dataset.save()

In [ ]:
session = fo.launch_app(dataset)
clear_output()

# Delete by

In [ ]:
! pip install tqdm

In [ ]:
from tqdm import tqdm

for sample in tqdm(dataset.iter_samples()):
    sample.tags==['delete']
# print(cnt)

## Sample Tag

In [ ]:
v1 = dataset.match_tags("delete")

In [ ]:
dataset.delete_samples(v1)

In [ ]:
dataset.save()
dataset

## Label Tag

In [ ]:
v1 = dataset.filter_labels('ground_truth',F('tags')==['tag_delete'])

In [ ]:
dataset.delete_samples(v1)

In [ ]:
dataset.save()
dataset